In [1]:
import numpy as np
from hmmlearn import hmm


In [2]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.1,0.9],[0.9,0.1]])
model.transmat_ = np.array([[0.9,0.1],[0.1,0.9]])
model.startprob_ = np.array([0.4,0.6])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [3]:
# model2 = hmm.CategoricalHMM(n_components=hmm_states,n_iter=1000,random_state=30,implementation='scaling')
# model2.fit(data.reshape(-1,1))
# hidden_states = model2.predict(data.reshape(-1,1))
# # print("Most likely hidden states:", hidden_states)


In [4]:
n_states = 2
n_features = 2
n_obs = data.shape[0]
# emission = np.random.rand(n_obs,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_features)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
emission = np.array([[0.2,0.8],[0.3,0.7]])

transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
transition = np.array([[0.6,0.4],[0.3,0.7]])


scale_factors = np.zeros((n_obs))
forward_hat = np.zeros((n_obs,n_states))

backward_hat = np.zeros((n_obs,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.01
max_iter = 100

mu = np.random.rand(n_states)

In [5]:
print(f'emission is {emission} and transition is {transition}')

emission is [[0.2 0.8]
 [0.3 0.7]] and transition is [[0.6 0.4]
 [0.3 0.7]]


In [6]:
for ite in range(max_iter):
    forward_hat = np.zeros((n_obs,n_states))
    backward_hat = np.zeros((n_obs,n_states))
    scale_factors = np.zeros((n_obs))


    forward_hat[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward_hat[0,:])
    forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
    
    for t in range(n_obs-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t+1]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')


    backward_hat[-1,:] = scale_factors[-1]
    for t in reversed(range(n_obs-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],transition.T)
        backward_hat[t,:] = temp/scale_factors[t]
        


    a = np.zeros((n_obs,n_states))
    b = np.zeros((n_obs,n_states,n_states))
    for i in range(n_obs):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
        temp = np.sum(a[i,:])
        a[i,:] = a[i,:]/temp

    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])
            # print(np.sum(b[0:-1,i,j]),np.sum(b[0:-1,i,:]))

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p},transition is {transition},emission is {emission},init is {init_prob}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    if p>p_old and p - p_old < tol:
        break
    p_old = p

p is:-818.8636434625259,transition is [[0.6004226  0.3995774 ]
 [0.29159828 0.70840172]],emission is [[0.38365454 0.61634546]
 [0.5636714  0.4363286 ]],init is [0.36281816 0.63718184]
p is:-687.1818910901122,transition is [[0.61661561 0.38338439]
 [0.28425633 0.71574367]],emission is [[0.3544178  0.6455822 ]
 [0.59941244 0.40058756]],init is [0.24945951 0.75054049]
p is:-680.9705606188808,transition is [[0.64600533 0.35399467]
 [0.27046388 0.72953612]],emission is [[0.30553344 0.69446656]
 [0.64000492 0.35999508]],init is [0.13578643 0.86421357]
p is:-667.8168121649195,transition is [[0.69678523 0.30321477]
 [0.24409724 0.75590276]],emission is [[0.24200889 0.75799111]
 [0.69710933 0.30289067]],init is [0.05017892 0.94982108]
p is:-639.8316110991913,transition is [[0.7741952  0.2258048 ]
 [0.19722176 0.80277824]],emission is [[0.17137898 0.82862102]
 [0.77066643 0.22933357]],init is [0.01008615 0.98991385]
p is:-593.4090016733306,transition is [[0.8615474  0.1384526 ]
 [0.13251788 0.86